## Veritabanı Bağlanti
![VeritabaniBaglanti.png](VeritabaniBaglanti.png)

* https://www.python.org/dev/peps/pep-0249/
* https://www.sqlite.org/download.html
* https://sqlitestudio.pl/
* https://www.connectionstrings.com/

In [1]:
import sqlite3

In [2]:
import os

os.makedirs("databases", exist_ok=True)

In [3]:
con = sqlite3.connect("./databases/mesajDB.db") #bu dosya veritaanı olarak kullanılacak.
#dir(con)

In [4]:
con.execute("""
CREATE TABLE mesaj
(
    Id int,
    Metin nvarchar(50)
)
""")

In [7]:
#tabloyu silmek için
#con.execute("DROP TABLE mesaj")

In [5]:
con.execute("INSERT INTO mesaj VALUES(1,'1. Kayıt')")
con.execute("INSERT INTO mesaj VALUES(2,'2. Kayıt')")
con.execute("INSERT INTO mesaj VALUES(3,'3. Kayıt')")
con.execute("INSERT INTO mesaj VALUES(4,'4. Kayıt')")
con.execute("INSERT INTO mesaj VALUES(5,'5. Kayıt')")

#
con.commit() #Auto commit yoksa commit edilmediğinde değişiklik oturum kapanınca geri alınır.
#con.rollback() # işlemi bilinçli şekilde iptal etmemizi sağlar.

In [6]:
'''
imlec = con.cursor()
imlec.execute("SELECT * FROM mesaj LIMIT 2")
'''

imlec=con.execute("SELECT * FROM mesaj LIMIT 2")

In [7]:
imlec.description #kolonlar hakkında bilgi

(('Id', None, None, None, None, None, None),
 ('Metin', None, None, None, None, None, None))

In [8]:
satirlar = imlec.fetchall()
satirlar

[(1, '1. Kayıt'), (2, '2. Kayıt')]

In [9]:
con.close()

### Comit ve Rollback

In [10]:
import sqlite3

con = sqlite3.connect("./databases/mesajDB.db")

In [11]:
con.execute("DELETE FROM mesaj WHERE Id IN (1,3)")

In [12]:
con.execute("SELECT * FROM mesaj").fetchall()

[(2, '2. Kayıt'), (4, '4. Kayıt'), (5, '5. Kayıt')]

In [13]:
con.rollback() #işlemler eri alınır

In [14]:
con.execute("SELECT * FROM mesaj").fetchall()

[(1, '1. Kayıt'),
 (2, '2. Kayıt'),
 (3, '3. Kayıt'),
 (4, '4. Kayıt'),
 (5, '5. Kayıt')]

### MSSQL veya Oracle ile Çalışmak
![MSSQLveOracleBaglanti.png](MSSQLveOracleBaglanti.png)

In [15]:
import pyodbc

In [16]:
pyodbc.drivers() #bilgisauarda yüklü olan kullanabileceğim driverları

['SQL Server',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'Oracle in OraDB19Home1',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [18]:
#Driver={ODBC Driver 17 for SQL Server};Server=myServerAddress;Database=myDataBase;Trusted_Connection=yes;
#Driver={ODBC Driver 17 for SQL Server};Server=myServerAddress;Database=myDataBase;UID=myUsername;PWD=myPassword;

db_config= {
            "Driver":"ODBC Driver 17 for SQL Server",
            "Server":"localhost",
            "Database":"master"
}

#{} üretmek için {{}} kullandık
conStr = "Driver={{{Driver}}};Server={Server};Database={Database};Trusted_Connection=yes;".format_map(db_config)

conStr

#
con = pyodbc.connect(conStr)

In [19]:
imlec=con.execute("SELECT @@version")

In [20]:
sonuc = imlec.fetchone() #tek hücre geliyor

In [21]:
print(sonuc[0]) #tuple ilk elemanı sonuc.

Microsoft SQL Server 2019 (RTM-CU25-GDR) (KB5036335) - 15.0.4360.2 (X64) 
	Mar 19 2024 00:23:01 
	Copyright (C) 2019 Microsoft Corporation
	Enterprise Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: ) (Hypervisor)



In [22]:
con.close()

### Pandas ile Tablodan DataFrame Üretmek

In [23]:
import sqlite3

con = sqlite3.connect("./databases/mesajDB.db")

In [24]:
import pandas as pd

df = pd.read_sql("SELECT * FROM mesaj",con)
type(df)

pandas.core.frame.DataFrame

In [28]:
df.head(3)
df.tail(3)

#
df.shape #5 satır 2 kolonluk bir matris
df.sample(3)

,Id,Metin
2,3,3. Kayıt
1,2,2. Kayıt
4,5,5. Kayıt


In [29]:
df.query("Id == 4")

,Id,Metin
3,4,4. Kayıt


In [31]:
df.Metin#.to_list()

0    1. Kayıt
1    2. Kayıt
2    3. Kayıt
3    4. Kayıt
4    5. Kayıt
Name: Metin, dtype: object

In [32]:
df.iloc[[2], [0,1]] #2. satır  0 ve 1. kolon
df.loc[0:5:2, ["Metin","Id"]]

,Metin,Id
0,1. Kayıt,1
2,3. Kayıt,3
4,5. Kayıt,5


In [33]:
df.isna().sum().sum()

0